In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive')

In [ ]:
!ls

 100k.csv
'academic transcript.gdoc'
'academic transcript.pdf'
'Amani Mashinani Presentation.pdf'
'Amani Mashinani Presentation.pptx'
 avataaars.png
'bash scripting and shell.txt'
'Blue Economy Entrance Document.pdf'
'blue economy essay 2 (1).docx'
'blue economy essay 2.docx'
'blue economy essay 2.gdoc'
'blue economy essay reveiwed real (1).pdf'
'Bobby Fischer Teaches Chess.pdf'
'BRCK covernote (1).docx'
'BRCK covernote.docx'
 cached
 Cancer
'C++ From Beginner to Beyond second June.pdf'
'Chess_ The Complete Guide To Chess - Master_ Chess Tactics, Chess Openings, and Chess Strategies ( PDFDrive ).pdf'
 CIC
'Colab Notebooks'
'CURRICULUM VITAE.docx'
'diploma certificate.pdf'
'Dracula (Bram Stoker) (z-lib.org).pdf'
'EEEI 00941 2015 EARTH SCIENCE ASSIGNMENT.gdoc'
'EEEI 414 - Computer Networks - Course Outline.pdf'
'Francis Macharia CV template (1).docx'
'Francis Macharia CV template (2).docx'
'Francis Macharia CV template.docx'
'Francis Macharia Data Engineer CV (1).gdoc'
'Francis Macharia 

In [ ]:
#importing the necessary libraries
import pandas as pd
#the use of the header 0 means that the first row is treated as the column name
df = pd.read_csv ('processed dialogue.csv',header=0)

In [ ]:
df.head()

,CHARACTER,LINE
0,OKOYE,You froze .
1,NAKIA,"Why are you here? Eesh, you ruined my mission !"
2,T'CHALLA,"My father is dead , Nakia."
3,T'CHALLA,I will be crowned King tomorrow and I wish for...
4,NAKIA,"Carry yourselves home now , and take the boy ...."


In [ ]:
#checking how many times the no of times a character has been mentioned.
#the sum is sth that is case sensitive and thus you need to make sure that the name is as it appears in the dataframe
sum(df['CHARACTER'] == "T'CHALLA")

120

In [ ]:
#checking the length of the dataframe
len(df)

699

In [ ]:
#the next part is the training is traing of the GPT 3 model(Generative Pretra,ined Transformer 3)
#since the gpt 3 is not a readily available library what you can use is pip install
!pip -q install transformers

     |████████████████████████████████| 4.2 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 


In [ ]:
# all the imports

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [ ]:
#sampling the dataset
df.sample(6)

,CHARACTER,LINE
59,CROWD,Damaku
118,T'CHALLA,Stop it. Stop it.
414,W'KABI,I'm afraid not Queen Mother
188,OKOYE,Someone did not get the memo. Definitely armed .
467,NAKIA,No. I save my country
45,KLAUE,Let's give the lady some space please .


In [ ]:
#if you need to rename the given columns
#df.rename(columns={'CHARACTER':'name','LINE':'line'}, inplace = True)
#the character name that the model is trained off is T'CHALLA
CHARACTER_NAME = "T'CHALLA"

In [ ]:
#the next part is the gold mine which looks the current line the character is speaking plus lines that precede the line
#list of lists contexted
contexted = []

# context window of size 7
n = 7
#this will forego line index 2 and 3 of T'CHALLA
for i in df[df.CHARACTER == CHARACTER_NAME].index:
  if i < n:
    continue
  row = []
  # we additionally substract 1, so row will contain current responces and 7 previous responces cz range is not inclusive of stop step
  prev = i - 1 - n  
  for j in range(i, prev, -1):
    row.append(df.LINE[j])
  contexted.append(row)


In [ ]:
#list of the columns that will be used in the context dataframe
columns = ['response', 'context']
columns = columns + ['context/' + str(i) for i in range(n - 1)]
#pandas.DataFrame.from_records>> creates a DataFrame from a structured ndarray, sequence of tuples or dicts
df1 = pd.DataFrame.from_records(contexted, columns=columns)

In [ ]:
df1.head(6)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,This never gets old .,Sister Nakia ... My Prince. We are home .,You will speak nothing of this day .,Thank you.,"Carry yourselves home now , and take the boy ....",I will be crowned King tomorrow and I wish for...,"My father is dead , Nakia.","Why are you here? Eesh, you ruined my mission !"
1,So surprised my little sister came to see me o...,Are you finished?,Like an antelope in headlights.,Did he freeze?,"Yes, general .",Take her to the River Province to prepare her...,"Thank you , Nakia. It is so good to have you b...",Queen Mother . . . Princess . My comfort for y...
2,"Update? No, it worked perfectly .","You wish . I'm here for the EMP beads, I've de...",So surprised my little sister came to see me o...,Are you finished?,Like an antelope in headlights.,Did he freeze?,"Yes, general .",Take her to the River Province to prepare her...
3,I cannot wait to see what kind of update you m...,"You are teaching me, what do you know?","How many times to I have to teach you, just be...","Update? No, it worked perfectly .","You wish . I'm here for the EMP beads, I've de...",So surprised my little sister came to see me o...,Are you finished?,Like an antelope in headlights.
4,"How are you feeling today, mama?",I cannot wait to see what kind of update you m...,"You are teaching me, what do you know?","How many times to I have to teach you, just be...","Update? No, it worked perfectly .","You wish . I'm here for the EMP beads, I've de...",So surprised my little sister came to see me o...,Are you finished?
5,I AM PRINCE T'CHALLA SO OF King NT'CHAKA!,Show him who you are !,"Where is your God, now?!No powers . No claws ....",Let the challenge begin!,It's challenge day .We have watched and listen...,M'Baku . What are you doing here?!,Yes .,Are they Jabari?


In [ ]:
#looking at the size of our data
len(df1)

118

In [ ]:
#splitting the dataset into training and test set through use of 
#from sklearn.model_selection import train_test_split
training_df1, val_df1= train_test_split(df1, test_size=0.1)
training_df1.head(10)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
87,Nol Zuri no! Zuri!! Noo!,"I'll take you both , Uncle James",I am the cause of your father's death. Not him...,Zuri don ' t !,Zuri,"Zuri , no",This is for my father,Get up
50,Murderer!,"Get the diamonds, quick !",I've been made,Hey!,Hey.,STAND DOWN!,It's now or never .,"I was going to buy a fancy suitcase, but I tho..."
84,I accept your challenge,"T'Challa, what do you know of this?",Weeks? I don't need weeks. The whole country a...,The challenge would take weeks to prepare,He has no rights here,As the son of N'Jobu he is within his rights,Do not do this T'Challa.,I'm exercising my blood right to challenge for...
85,"This is your last chance, throw down your weap...",I locked him in the office . He's not going an...,Where is Agent Ross?,The King will now have the strength of the Bla...,He will never beat T' Challa . He should have...,I accept your challenge,"T'Challa, what do you know of this?",Weeks? I don't need weeks. The whole country a...
96,Thank you,Could you give me and Lord M' Baku a moment?,Are you done? Are you - are you done?,What? You're gonna need all the help you can get,I'm in too,As will I,The Black Panther lives . And when he fights f...,"If he gets control of our technology , no wher..."
22,But waging war on other countries has never be...,"You let refugees in , they bring their problem...",Foreign aid ... refugee programs.,More like what?,Nakia thinks we should be doing more .,Of course.I see Nakia is back ... you guys goi...,"Is that him? Glory to Bast, man, is he still g...",if that's what I wanted !
86,"Zuri , no",This is for my father,Get up,Snap out of it T'Challa!,Yield !,Let the challenge begin,"I killed in America, Afghanistan,Iraq . I took...",I lived my entire life waiting for this moment...
76,Did he reveal anything about his identity?,Now these guys are serious . They will drop of...,Eric Stevens . Graduated Annapolis age 19 . MI...,"Brother , you need to get here now.",Hey . Look at me. You can't let your father ' ...,He didn ' t even give him a proper burial ... ...,No man is perfect . Not even your father.,He killed his own brother ... and left a child...
73,No no no no no no . . .,He helped Klaue steal the vibranium,No .,"He knew your father would not support this , s...",I observed for as long as I could !Their leade...,Your uncle took a War Dog assignment in Americ...,I am your King now!,I promised the King to say nothing .
79,And what tools are those?,Y'all sittin' up here comfortable.Must feel go...,My goodness,I want the throne,"I don't care that you brought Klaue, the only ...",I'm standing in your house . Serving justice t...,Speak,Speak.


In [ ]:
# create dataset suitable for our model
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df1, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df1.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df1_training, df1_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df1_val if evaluate else df1_training)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

BUILDING THE MODEL BASED ON  MICROSOFT PRETRAINED GPT DialoGPT

In [ ]:
from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
import torch
#tokenizers
#tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
#tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")



#models
#model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-medium")
#model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-large")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/823M [00:00<?, ?B/s]

In [ ]:
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
# Args to allow for easy convertion of python script to notebook
#one may look at them as hyperparameters
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        #self.output_dir = 'output-medium'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-medium'
        #self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-medium'
        #self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-medium'
        #self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        #no of samples model will see before updating gradient
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        #the no of times the model will cycle through the data
        self.num_train_epochs = 16
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df1_training, df1_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df1_training, df1_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# Main runner

def main(df1_training, df1_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df1_training, df1_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df1_training, df1_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(training_df1, val_df1)

05/30/2022 10:05:03 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
05/30/2022 10:05:32 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7f371e669110>
05/30/2022 10:05:32 - INFO - __main__ -   Creating features from dataset file at cached
05/30/2022 10:05:32 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTo

Epoch:   0%|          | 0/16 [00:00<?, ?it/s]

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

05/30/2022 10:09:47 - INFO - __main__ -    global_step = 416, average loss = 0.9212880359711841
05/30/2022 10:09:47 - INFO - __main__ -   Saving model checkpoint to output-small
05/30/2022 10:10:00 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small']
05/30/2022 10:10:11 - INFO - __main__ -   Creating features from dataset file at cached
05/30/2022 10:10:11 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
05/30/2022 10:10:11 - INFO - __main__ -   ***** Running evaluation  *****
05/30/2022 10:10:11 - INFO - __main__ -     Num examples = 12
05/30/2022 10:10:11 - INFO - __main__ -     Batch size = 4


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

05/30/2022 10:10:11 - INFO - __main__ -   ***** Eval results  *****
05/30/2022 10:10:11 - INFO - __main__ -     perplexity = tensor(3.8927)


{'perplexity_': tensor(3.8927)}

perplexity = tensor(4.1062)
{'perplexity_': tensor(4.1062)}
#the perplexity tensor shows how confused the model is on which words to choose, a larger one means bad news

Load the Trained Model

In [ ]:
#tokens
#tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
#tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-large')


#models
model = AutoModelWithLMHead.from_pretrained('output-small')
#model = AutoModelWithLMHead.from_pretrained('output-medium')
#model = AutoModelWithLMHead.from_pretrained('output-large')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
# Let's chat for 4 lines
for step in range(6):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature=0.8
    )
    
    # pretty print last ouput tokens from bot
    print("T'ChallaBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hello?
T'ChallaBot: Hello. How are you?
>> User:Can I come home?
T'ChallaBot: Of course.You can stay.
>> User:How about world peace?
T'ChallaBot: That will be nice.
>> User:Okay then I agree
T'ChallaBot: Speak.
>> User:How was your day?
T'ChallaBot: It was good.You too.
>> User:Mine was a bit terrible
T'ChallaBot: That's good.
